In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_rows', None)



In [5]:
TEST = 0.2
LAMBDA = 0.04
LR = 0.01
EPOCHS = 200
BATCH_SIZE = 64

In [6]:
raw_data = pd.read_csv("housing.csv")
raw_data.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


In [7]:
print(raw_data["median_house_value"].mean())
raw_data["median_house_value"].std()

206855.81690891474


115395.61587441387

In [8]:
for key in raw_data .keys():
    nan_values = raw_data[key].isna().any()
    print(f"raw_data[{key}] nan_values is {nan_values}")

nan_values = raw_data["total_bedrooms"].isna()
raw_data["total_bedrooms"][nan_values] = raw_data["total_bedrooms"].mean()

raw_data[longitude] nan_values is False
raw_data[latitude] nan_values is False
raw_data[housing_median_age] nan_values is False
raw_data[total_rooms] nan_values is False
raw_data[total_bedrooms] nan_values is True
raw_data[population] nan_values is False
raw_data[households] nan_values is False
raw_data[median_income] nan_values is False
raw_data[median_house_value] nan_values is False
raw_data[ocean_proximity] nan_values is False


C:\Users\ramze\AppData\Local\Temp\ipykernel_23164\3177983568.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data["total_bedrooms"][nan_values] = raw_data["total_bedrooms"].mean()


In [9]:
for key in raw_data.keys():
    nan_values = raw_data[key].isna().any()
    print(f"raw_data[{key}] nan_values is {nan_values}")

nan_values = raw_data.isna().any().any()

print("--"*30+f"\nis there nan values in raw_data :{nan_values}")

raw_data[longitude] nan_values is False
raw_data[latitude] nan_values is False
raw_data[housing_median_age] nan_values is False
raw_data[total_rooms] nan_values is False
raw_data[total_bedrooms] nan_values is False
raw_data[population] nan_values is False
raw_data[households] nan_values is False
raw_data[median_income] nan_values is False
raw_data[median_house_value] nan_values is False
raw_data[ocean_proximity] nan_values is False
------------------------------------------------------------
is there nan values in raw_data :False


In [10]:
raw_data.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,419.266592,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,297.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,438.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,643.250000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [11]:
ocean = sorted(raw_data["ocean_proximity"].unique())
ocean_idx = {name:idx for idx,name in enumerate(ocean)}
print(ocean_idx)

raw_data["ocean_proximity"] = raw_data["ocean_proximity"].apply(lambda name:ocean_idx[name])
raw_data.head()

{'<1H OCEAN': 0, 'INLAND': 1, 'ISLAND': 2, 'NEAR BAY': 3, 'NEAR OCEAN': 4}


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,3
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,3
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,3
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,3
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,3


In [12]:
labels = raw_data["median_house_value"]
data = raw_data.drop("median_house_value",axis=1)

data=(data-data.mean())/data.std()


train_data,test_data=train_test_split(data,test_size=TEST,random_state=23)
train_labels,test_lebels=train_test_split(labels,test_size=TEST,random_state=23)

print(f"train shape :{train_data.shape}  train labels :{train_labels.shape}")
print(f"test shape  :{test_data.shape}   test labels :{test_lebels.shape}")


train shape :(16512, 9)  train labels :(16512,)
test shape  :(4128, 9)   test labels :(4128,)


# GradientBoosting

In [13]:
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor


model = GradientBoostingRegressor(n_estimators=4000,learning_rate=0.01,max_depth=11,subsample=0.5,alpha=0.25)
regressor = model.fit(X=train_data,y=train_labels.values)
pred = regressor.predict(test_data.values)

print(f"Gradient boosting RMSE is {np.sqrt(mean_squared_error(test_lebels.values,pred))}")


c:\dev\python\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


Gradient boosting RMSE is 44305.43809208242


# Random Forest

In [14]:
model = RandomForestRegressor()
regressor = model.fit(X=train_data,y=train_labels.values)
pred = regressor.predict(test_data.values)

print(f"RandomForest RMSE is {np.sqrt(mean_squared_error(test_lebels.values,pred))}")

RandomForest RMSE is 48108.95378087554


c:\dev\python\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


# Deep learning

In [15]:

train_data = tf.cast(train_data.values,dtype=tf.float32)
test_data = tf.cast(test_data.values,dtype=tf.float32)

train_labels = tf.cast(train_labels.values,dtype=tf.float32)
test_labels = tf.cast(test_lebels.values,dtype=tf.float32)


class Network(tf.keras.Model):
    def __init__(self):
        super(Network,self).__init__()
        self.fc1 = tf.keras.layers.Dense(128,activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(LAMBDA))
        self.fc2 = tf.keras.layers.Dense(128,activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(LAMBDA))
        self.fc3 = tf.keras.layers.Dense(64,activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(LAMBDA))
        self.fc4 = tf.keras.layers.Dense(64,activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(LAMBDA))
        self.drop = tf.keras.layers.Dropout(0.4)
        self.fc5 = tf.keras.layers.Dense(1,activation="linear")
    def call(self,inputs):
        x = self.fc1(inputs)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.drop(x)
        x = self.fc5(x)
        return x
    
model = Network()

model.compile(
    optimizer=tf.keras.optimizers.Adam(LR),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=["mean_squared_error"]
)



In [16]:
model.fit(x=train_data,y=train_labels,batch_size=BATCH_SIZE,epochs=EPOCHS,verbose=2)

Epoch 1/200
258/258 - 1s - loss: 12375137280.0000 - mean_squared_error: 12375137280.0000 - 1s/epoch - 5ms/step
Epoch 2/200
258/258 - 0s - loss: 5561462784.0000 - mean_squared_error: 5561462784.0000 - 371ms/epoch - 1ms/step
Epoch 3/200
258/258 - 0s - loss: 5401814528.0000 - mean_squared_error: 5401814528.0000 - 346ms/epoch - 1ms/step
Epoch 4/200
258/258 - 0s - loss: 5385400320.0000 - mean_squared_error: 5385400320.0000 - 251ms/epoch - 973us/step
Epoch 5/200
258/258 - 0s - loss: 5416591360.0000 - mean_squared_error: 5416591360.0000 - 249ms/epoch - 967us/step
Epoch 6/200
258/258 - 0s - loss: 5310359552.0000 - mean_squared_error: 5310359552.0000 - 247ms/epoch - 956us/step
Epoch 7/200
258/258 - 0s - loss: 5129598464.0000 - mean_squared_error: 5129598464.0000 - 263ms/epoch - 1ms/step
Epoch 8/200
258/258 - 0s - loss: 5086001152.0000 - mean_squared_error: 5086001152.0000 - 243ms/epoch - 943us/step
Epoch 9/200
258/258 - 0s - loss: 5172289024.0000 - mean_squared_error: 5172289024.0000 - 257ms/ep

In [19]:


pred_train= model.predict(train_data)
print(np.sqrt(mean_squared_error(train_labels,pred_train)))

pred= model.predict(test_data)
print(f"Neural Network RMSE is {np.sqrt(mean_squared_error(test_labels,pred))}")

516/516 [==============================] - 0s 799us/step
42926.04
129/129 [==============================] - 0s 789us/step
Neural Network RMSE is 51602.50390625
